In [1]:
from models import AudioEmbeddingNet

In [2]:
inp = './examples/utt0.wav'

In [3]:
model = AudioEmbeddingNet()

Using cache found in /Users/siddhantpathak/.cache/torch/hub/harritaylor_torchvggish_master


In [4]:
out = model(inp)

In [6]:
out.shape

torch.Size([1, 384])